In [1]:
from edgar_functions import *
from scorecard_naming import (
    Balance_sheet_mapping,
    Income_statement_mapping,
    Cash_flow_mapping,
)

ticker = "ABT"

In [2]:
# print_links_to_desired_statment(ticker, 'income_statement')
# print_links_to_desired_statment(ticker, 'balance_sheet')
# print_links_to_desired_statment(ticker, 'cash_flow_statement')
# balance = form_statement_for_all_available_years(ticker, 'balance_sheet')
# income = form_statement_for_all_available_years(ticker, 'income_statement')
# cash = form_statement_for_all_available_years(ticker, 'cash_flow_statement')
# save_dataframe_to_csv(balance, 'statements', 'temp', 'balance_sheet', 'annual')
# save_dataframe_to_csv(income, 'statements', 'temp', 'income_statement', 'annual')
# save_dataframe_to_csv(cash, 'statements', 'temp', 'cash_flow', 'annual')

In [3]:
balance = pd.read_csv("statements/temp/balance_sheet_annual.csv", index_col=0)
income = pd.read_csv("statements/temp/income_statement_annual.csv", index_col=0)
cash = pd.read_csv("statements/temp/cash_flow_annual.csv", index_col=0)

In [4]:
balance_sheet = balance
income_statement = income
cash_flow_statement = cash

In [5]:
statement = income_statement
statement_mapping = Income_statement_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31
us-gaap_IncomeStatementAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net Revenue,43653.00,43075.00,34608.00,31904.00,30578.00,27390.00,20853.00,NaN,NaN
Cost of Goods Sold,19142.00,18537.00,15003.00,13231.00,12706.00,12409.00,9094.00,NaN,NaN
Amortization of Intangible Assets,2013.00,2047.00,2132.00,NaN,NaN,NaN,NaN,NaN,NaN
Research and Development,2888.00,2742.00,2420.00,2440.00,2300.00,2260.00,NaN,NaN,NaN
SG&A,11248.00,11324.00,9696.00,9765.00,9744.00,9182.00,6736.00,6785.00,6530.00
Operating Expenses,35291.00,34650.00,29251.00,27372.00,26928.00,25826.00,17827.00,17538.00,17648.00
Operating Income,8362.00,8425.00,5357.00,4532.00,3650.00,1564.00,3026.00,2867.00,2599.00
Interest Expense,558.00,533.00,546.00,670.00,826.00,904.00,431.00,163.00,150.00
Interest Income,-183.00,-43.00,-46.00,-94.00,-105.00,-124.00,-99.00,-105.00,-77.00


In [6]:
statement = balance_sheet
statement_mapping = Balance_sheet_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31
us-gaap_AssetsCurrentAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash,9882.0,9799.0,6838.0,3860.0,3844.0,9407.0,18620.0,5001.0
Marketable Securities,288.0,450.0,310.0,280.0,242.0,203.0,155.0,1124.0
Accounts Receivable,6218.0,6487.0,6414.0,5425.0,5182.0,5249.0,3248.0,3418.0
us-gaap_InventoryNetAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_InventoryFinishedGoodsNetOfReserves,3805.0,3081.0,3030.0,2784.0,2407.0,2339.0,1624.0,1744.0
us-gaap_InventoryWorkInProcessNetOfReserves,680.0,694.0,712.0,560.0,499.0,472.0,294.0,316.0
us-gaap_InventoryRawMaterialsNetOfReserves,1688.0,1382.0,1270.0,972.0,890.0,790.0,516.0,539.0
Inventory,6173.0,5157.0,5012.0,4316.0,3796.0,3601.0,2434.0,2599.0
abt_PrepaidExpenseAndOtherReceivables,2663.0,2346.0,1867.0,1786.0,1568.0,1667.0,NaN,NaN


In [8]:
statement = cash_flow_statement
statement_mapping = Cash_flow_mapping
flattened_mapping = {old: new for new, old in statement_mapping.items() for old in old}
renamed_statement = statement.rename(index=flattened_mapping)
renamed_statement

,2022-12-31,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31
us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Net Earnings,6933.0,7071.0,4495.0,NaN,NaN,NaN,1400.0,4423.0,2284.0
us-gaap_AdjustmentsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivitiesAbstract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,1254.0,1491.0,1195.0,1078.0,1100.0,1046.0,803.0,871.0,918.0
Amortization,2013.0,2047.0,2132.0,1936.0,2178.0,1975.0,550.0,601.0,630.0
...,...,...,...,...,...,...,...,...,...
us-gaap_EquityMethodInvestmentOtherThanTemporaryImpairment,NaN,NaN,NaN,NaN,NaN,NaN,947.0,NaN,NaN
us-gaap_GainLossOnSaleOfEquityInvestments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-207.0,NaN
us-gaap_ProceedsFromSaleOfEquityMethodInvestments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2290.0,NaN
Cash Paid for Income Tax Net,NaN,NaN,NaN,NaN,NaN,NaN,620.0,631.0,448.0
